In [1]:
import numpy as np
import tensorflow as tf
import time

2024-04-23 16:54:45.909869: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 16:54:45.909946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 16:54:46.806216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 16:54:48.928837: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 16:55:13.128133: W tensorflow/compiler/tf2

In [3]:
def base_conv():
    model = tf.keras.models.Sequential([

        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
        tf.keras.layers.GlobalAveragePooling2D(),
        
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

def create_model():
    # Load MNIST dataset

    model = base_conv()

    # Compile the model
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    model.summary()
    # Save the trained model
    model.save("models/mnist_cnn.keras")
    print('saved model')
    return model

create_model()

2024-04-23 16:56:33.946699: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 depthwise_conv2d (Depthwis  (None, 220, 220, 32)      320       
 eConv2D)                                                        
                                                                 
 conv2d_1 (Conv2D)           (None, 218, 218, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 109, 109, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 107, 107, 64)      36928     
                                                                 
 depthwise_conv2d_1 (Depthw  (None, 105, 105, 64)      6

In [15]:

def conv_1a(input_size):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_size),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
        tf.keras.layers.GlobalAveragePooling2D(),
        
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
        
        
    ])
    model.build(tf.random.normal(input_size))
    return model

model = conv_1a((224, 224, 3))
model.save("models/GPU_CNN_4.keras")



In [9]:
def conv_1b(input_size):
    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Dense(128, input_shape=input_size, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.build(tf.random.normal(input_size))
    return model

#convert one model to tflite
def convert_to_tflite(model, input_size, n):
    def representative_dataset():
        for _ in range(100):
            yield [tf.random.normal([1, *input_size])]

    quantizer = tf.lite.TFLiteConverter.from_keras_model(model)
    quantizer.optimizations = [tf.lite.Optimize.DEFAULT]
    quantizer.representative_dataset = representative_dataset
    quantizer.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
    quantizer.inference_input_type = tf.uint8  # or tf.uint8
    quantizer.inference_output_type = tf.uint8  # or tf.uint8
    
    tflite_quant_model = quantizer.convert()

    # Save the TFLite model to a file
    with open(f"models/TPU_CNN_{n}", "wb") as f:
        f.write(tflite_quant_model)
    print("saved tf lite model")
    return tflite_quant_model

input_size = (256,) 
model_tflite = conv_1b(input_size)
model_tflite = convert_to_tflite(model_tflite, input_size, 3)

INFO:tensorflow:Assets written to: /tmp/tmpke5k1ct9/assets


INFO:tensorflow:Assets written to: /tmp/tmpke5k1ct9/assets
/u/pj8wfq/rl/MC/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


saved tf lite model


2024-04-23 17:09:01.982503: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-23 17:09:01.982528: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-23 17:09:01.982740: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpke5k1ct9
2024-04-23 17:09:01.983685: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-23 17:09:01.983701: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpke5k1ct9
2024-04-23 17:09:01.985829: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-04-23 17:09:02.003233: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpke5k1ct9
2024-04-23 17:09:02.010422: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 27684 m

In [ ]:
n_samples = 10000
def generate_input(n):
    return np.random.rand(n, 28, 28).astype(np.float32)
#load model 1

model_1a = tf.keras.models.load_model("models/mnist_dnn1a.keras")
start = time.perf_counter()
outputs = model_1a.predict(generate_input(n_samples), verbose=0)
print(outputs.shape)
end = time.perf_counter()
print("average inference time: ", (end - start)*1000/n_samples, " ms")


In [ ]:

def generate_1b_input():
    return np.random.rand(1,128).astype(np.float32)

def tflite_inference(num_trials):
    
    #load model 1
    model_1a = tf.keras.models.load_model("models/mnist_dnn1a.keras")
    
    
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path="mnist_model_batched.tflite")
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    output_scale, output_zero_point = output_details[0]['quantization']
    input_scale, input_zero_point = input_details[0]["quantization"]
    
    # Prepare input data
    input_shape = input_details[0]['shape']
    inference_times = []
    inputs = [generate_1b_input() for _ in range(num_trials)]
    for input_data in inputs:
        start = time.perf_counter()
        input_data = (input_data / input_scale) + input_zero_point
        interpreter.set_tensor(input_details[0]['index'], input_data.astype(np.uint8))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        output_data = output_scale * (output_data - output_zero_point)
        end = time.perf_counter()
        inference_times.append((end - start)*1000)

    avg_inference_time = np.mean(inference_times)
    print("average inference time: ", avg_inference_time, "ms")

tflite_inference(10)